# Fallback models

In many cases it is good to have a fallback model. There are solution like [LiteLLM](), an LLM gateway that allows you to configure accesses, fallback models, quotas and much more.

If OpenAI API key is already exported, you can install `litellm[proxy]` and invoke, for example

```
litellm --model gpt-3.5-turbo
```

You will have it running immediately for testing the connection. Then, you can simply invoke the library and connect your agent to the endpoint that redirects the traffic.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat = ChatOpenAI(
    openai_api_base="http://0.0.0.0:4000", # set openai_api_base to the LiteLLM Proxy
    model = "gpt-3.5-turbo",
    temperature=0.1
)

messages = [
    SystemMessage(
        content="You are a helpful assistant that im using to make a test request to."
    ),
    HumanMessage(
        content="test from litellm. tell me why it's amazing in 1 sentence"
    ),
]
response = chat(messages)

print(response)

### Locally

You can also try with locally deployed models. Using solutions like Ollama or [vLLM](https://docs.vllm.ai/en/stable/index.html) you can easily host and serve you own models. Although, machine specifications may be higher than the usual laptop, going for higher RAM and a little bit of GPU if possible.

Feel free to check it out.

In [ ]:
from langchain_community.llms import VLLM

llm = VLLM(
    model="TheBloke/Llama-2-7b-Chat-AWQ",
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=128,
    vllm_kwargs={"quantization": "awq"}, # quantized
    top_k=10,
    top_p=0.95,
    temperature=0.8,
)

print(llm.invoke("What is the capital of France ?"))